Model download link: https://github.com/OlafenwaMoses/ImageAI/releases/download/essentials-v5/resnet50_coco_best_v2.1.0.h5/

In [67]:
from imageai.Detection import ObjectDetection
import os

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("models/resnet50_coco_best_v2.1.0.h5")
detector.loadModel()

In [ ]:
detections_dict = dict()
dir_name = "images"
images_left = len(os.listdir(dir_name))

for filename in os.listdir(dir_name):
    detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename, output_type="array")[1]
    # detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename,
    #                                              output_image_path=f"{dir_name}/new_" + filename)

    detections_dict[filename] = []

    for eachObject in detections:
        detections_dict[filename].append((eachObject["name"], eachObject["percentage_probability"]))
        # print(filename, (eachObject["name"], eachObject["percentage_probability"]))

    images_left -= 1
    print("images left:", images_left)

images left: 20850
images left: 20849
images left: 20848
images left: 20847
images left: 20846
images left: 20845
images left: 20844
images left: 20843
images left: 20842
images left: 20841
images left: 20840
images left: 20839
images left: 20838
images left: 20837
images left: 20836
images left: 20835
images left: 20834
images left: 20833
images left: 20832
images left: 20831
images left: 20830
images left: 20829
images left: 20828
images left: 20827
images left: 20826
images left: 20825
images left: 20824
images left: 20823
images left: 20822
images left: 20821
images left: 20820
images left: 20819
images left: 20818
images left: 20817
images left: 20816
images left: 20815
images left: 20814
images left: 20813
images left: 20812
images left: 20811
images left: 20810
images left: 20809
images left: 20808
images left: 20807
images left: 20806
images left: 20805
images left: 20804
images left: 20803
images left: 20802
images left: 20801
images left: 20800
images left: 20799
images left:

In [ ]:
import pickle

with open("images_objects.pkl", "wb") as f:
    pickle.dump(detections_dict, f)

In [ ]:
with open("images_objects.pkl", "rb") as f:
    print(pickle.load(f))